In [1]:
import graphviz
import re

In [2]:
epsilon = 'ε'
end = '$'
operators = ['*', '|', '+', '?', '.', '(', ')']
symbols = []

In [3]:
def infix_to_postfix(reg):
    pattern = r'([a-zA-Z]|[0-9]|\*|\?|\))(?=(\(|[a-zA-Z]|[0-9]))'
    exp = re.sub(pattern, r'\1.', reg.replace(" ", ""))

    precedence = {'+': 1, '|': 1, '.': 2, '*': 3, '?': 3}
    postfix = []
    operator_stack = []

    for token in exp:
        if token.isalnum():
            postfix.append(token)
        elif token == '(':
            operator_stack.append(token)
        elif token == ')':
            while operator_stack and operator_stack[-1] != '(':
                postfix.append(operator_stack.pop())
            operator_stack.pop()
        elif token in operators:
            while operator_stack and operator_stack[-1] != '(' and precedence[operator_stack[-1]] >= precedence[token]:
                postfix.append(operator_stack.pop())
            operator_stack.append(token)

    while operator_stack:
        postfix.append(operator_stack.pop())

    return "".join(postfix)

In [4]:
class State:
    count = 0
    
    def __init__(self, label=None):
        self.transitions = {}
        self.label = label if label is not None else f"q{State.count}"
        self.final = False
        self.contained_states = set([])
        State.count += 1
    
    def add_transition(self, symbol, to_state):
        self.transitions.setdefault(str(symbol), []).append(to_state)
    
    def show(self):
        print("Label: " + self.label)
        print("Final: " + str(self.final))
        print("Transitions:")
        for sym, tr in self.transitions.items():
            print(f"\t{sym}: " + str([st.label for st in tr]))
    
    def visualize(self, filename):
        dot = graphviz.Digraph(format='png')
        
        stack = [self]
        visited = set()
        
        while stack:
            state = stack.pop()
            if state not in visited:
                visited.add(state)
                dot.node(state.label, state.label, shape='doublecircle' if state.final else 'circle')
                for label, targets in state.transitions.items():
                    for st in targets:
                        dot.edge(state.label, st.label, label=label)
                        stack.append(st)
        dot.node("start", "", shape='point')
        dot.edge("start", self.label)
        dot.render(filename, cleanup=True, view=True)

In [5]:
class NFA:
    def __init__(self) -> None:
        self.start = State()
        self.states = [self.start]
    
    def regex_to_nfa(self, regex:str):
        regex = infix_to_postfix(regex)
        states_map = {}
        inds = []

        for i, token in enumerate(regex):
            if token in symbols:
                st1 = State()
                st2 = State()
                st1.add_transition(token, st2)
                states_map[i] = [st1, st2]
                self.states.append(st1)
                self.states.append(st2)
            else:
                if token == '+' or token == '|':
                    low = inds[-2]
                    high = inds[-1]
                    st1 = State()
                    st2 = State()
                    st1.add_transition(epsilon, states_map[low][0])
                    st1.add_transition(epsilon, states_map[high][0])
                    states_map[low][1].add_transition(epsilon, st2)
                    states_map[high][1].add_transition(epsilon, st2)
                    states_map[i] = [st1, st2]
                    inds.remove(low)
                    inds.remove(high)
                    self.states.append(st1)
                    self.states.append(st2)
                elif token == '.':
                    low = inds[-2]
                    high = inds[-1]
                    states_map[low][1].add_transition(epsilon, states_map[high][0])
                    states_map[i] = [states_map[low][0], states_map[high][1]]
                    inds.remove(low)
                    inds.remove(high)
                elif token == '*':
                    high = inds[-1]
                    states_map[high][1].add_transition(epsilon, states_map[high][0])
                    st = State()
                    st.add_transition(epsilon, states_map[high][0])
                    states_map[i] = [st, states_map[high][0]]
                    inds.remove(high)
                    self.states.append(st)
                elif token == '?':
                    high = inds[-1]
                    states_map[high][1].add_transition(epsilon, states_map[high][0])
                    st = State()
                    st.add_transition(epsilon, states_map[high][0])
                    states_map[i] = [st, states_map[high][1]]
                    inds.remove(high)
                    self.states.append(st)
            inds.append(i)
        
        self.start.add_transition(epsilon, states_map[inds[-1]][0])
        states_map[inds[-1]][1].final = True

    def visualize_nfa(self, filename):
        self.start.visualize(filename)

In [6]:
class DFA:
    def __init__(self) -> None:
        self.start = None
        self.dead = State("q'")
        self.states = set([])

    def convert_nfa_to_dfa(self, nfa:NFA):
        self.start = nfa.start
        closure = self.__epsilon_closure(nfa)
        
        for state, cls in closure.items():
            for sym in symbols:
                final_tr = []
                for st in list(cls):
                    if st.final:
                        state.final = True
                    trs = st.transitions.get(sym, [])
                    final_tr.extend(trs)
                state.transitions[sym] = list(set(final_tr))
                if epsilon in state.transitions:
                    del state.transitions[epsilon]
        

        # self.visualize_dfa("output/nfa")
        stack = [self.start]
        states = []
        states_vals = []
        while stack:
            current = stack.pop(0)
            self.states.add(current)
            for sym in symbols:
                trs = []
                for st in set(list(current.contained_states) + [current]):
                    trs.extend(st.transitions.get(sym, []))
                trs = set(trs)
                if len(trs) == 0:
                    # current.transitions[sym] = [self.dead]
                    continue
                elif trs not in states:
                    new_state = State()
                    new_state.contained_states = trs
                    current.transitions[sym] = [new_state]
                    states.append(trs)
                    states_vals.append(new_state)
                    stack.append(new_state)
                else:
                    current.transitions[sym] = [states_vals[states.index(trs)]]
        
        # Final States
        for st in list(self.states):
            finals = [s.final for s in st.contained_states] + [st.final]
            if any(finals):
                st.final = True

    def __epsilon_closure(self, nfa:NFA):
        closure = {}
        
        for state in nfa.states:
            cls = set()
            stack = [state]
            while stack:
                st = stack.pop()
                if st not in cls:
                    cls.add(st)
                    stack.extend(st.transitions.get(epsilon, []))
            closure[state] = cls
        return closure
    
    def visualize_dfa(self, filename):
        self.start.visualize(filename)

In [7]:
regex = "d*(a?+b)2*"
symbols = list(set([c for c in regex if c.isalnum()]))
nfa = NFA()
nfa.regex_to_nfa(regex)
nfa.visualize_nfa("output/nfa")

In [8]:
dfa = DFA()
dfa.convert_nfa_to_dfa(nfa)
dfa.visualize_dfa("output/dfa")